# 使用python3运行，解决encoding的问题

In [1]:
import ipywidgets as widgets
from IPython.display import display

from config import new_mysql_connect
from link_name import IDNameLinker, IDNameSelector

db = new_mysql_connect()
cursor = db.cursor()

# 准备原始数据

In [2]:
#cursor.execute('select distinct uname from allspark_dev.uniscore_16 where uid is null and uname not in (select verbose_name from datamodel.university_name_mapping)')
cursor.execute('select distinct uname from allspark_dev.uniscore_16 where uid is null')
results = cursor.fetchall() 
raw_datas = [ row[0] for row in results ]
raw_count = len(raw_datas)

# 准备标准数据

In [3]:
uname_selector = IDNameSelector('sdb.sdb_university', 'id', 'name')
names = uname_selector.get_all()

In [4]:
import re
class NameLinker(object):

    def __init__(self, standards):
        self._db = standards
        self._name_patterns = []
        self._name_patterns.append(re.compile('([^()]+)\(引进[^()]+课程\)'))
        self._name_patterns.append(re.compile('([^()]+)\([^()]+专业\)'))
        self._name_patterns.append(re.compile('([^()]+)\(与[^()]+合办\)'))
        self._name_patterns.append(re.compile('([^()]+)\(与[^()]+联办\)'))
        self._name_patterns.append(re.compile('([^()]+)\([^()]*就读[^()]*\)'))
        self._name_patterns.append(re.compile('([^()]+)\([^()]*办学[^()]*\)'))
        
        self._split_pattern = re.compile('([^()]+)\(原([^()]+)\)')

    def get(self, name):
        result = self.get_equal_value(name)
        if result:
            return result    
        for new_name in self.correct_name(name):
            tmp = self.get_equal_value(new_name)
            if tmp:
                if result and result['id'] != tmp['id']:
                    return
                result = tmp
        return result
        
    def get_equal_value(self, name):
        for key, value in self._db.items():
            if value == name:
                return {'id': key, 'name': value}

    def correct_name(self, name):
        names = set()     
        # 第一步：标准化名称
        name = name.replace('（', '(').replace('）', ')')
        name = name.replace('(较高收费)', '').replace('(较高学费)', '')
        name = name.replace('(中外合作办学)', '').replace('(对外合作办学)', '')
        name = name.replace('(仅招少数民族)', '')
        name = name.replace('(护理)', '')
        
        # 补齐（）
        if name.count('(') - name.count(')') == 1:
            name = name + ')'
        
        # 正则修正
        for pattern in self._name_patterns:        
            matches = pattern.match(name)
            if matches:
                name = matches.group(1)
        names.add(name)
        
        # 第二步：拆出可能的名称
        matches = self._split_pattern.match(name)
        if matches:
            names.add(matches.group(1))
            names.add(matches.group(2))
        
        return names

linker = NameLinker(names)

# 开始执行possible_values替换

In [5]:
results = []
for name in raw_datas:
    value = linker.get(name)
    results.append({'name': name, 'value':value})
results_no = [ item for item in results if item['value'] is None ]
results_ok = [ item for item in results if item['value'] is not None ]

print('正常匹配：', len(results_ok))
print('没有匹配：', len(results_no))

正常匹配： 0
没有匹配： 326


In [6]:
def _fn(e):
    print('共有', raw_count, '个待映射的名称')
    for i, name in enumerate(raw_datas):
        print('[{}/{}]: {}'.format(i+1, raw_count, name))
btn = widgets.Button(description='显示待处理的院校名称')
btn.on_click(_fn)
display(btn)

def _fn(e):
    print('共有', len(results_ok), '个待映射的名称')
    for i, name in enumerate(results_ok):
        item = results_ok[i]
        print('[{}/{}]: {} -> {}'.format(i+1, len(results_ok), item['name'], item['value']))
btn = widgets.Button(description='显示完成处理的院校名称')
btn.on_click(_fn)
display(btn)

def _fn(e):
    print('共有', len(results_no), '个待映射的名称')
    for i, name in enumerate(results_no):
        item = results_no[i]
        print('[{}/{}]: {}'.format(i+1, len(results_no), item['name']))
btn = widgets.Button(description='显示未完成处理的院校名称')
btn.on_click(_fn)
display(btn)

def _fn(e):
    print('更新映射内容到数据库中')
    for i, name in enumerate(results_ok):
        item = results_ok[i]
        try:
            cursor.execute('insert into datamodel.university_name_mapping(verbose_name, standard_name, standard_id) values(%s, %s, %s)', [item['name'], item['value']['name'], item['value']['id']])
            print('[{}/{}]: {} -> {}'.format(i+1, len(results_ok), item['name'], item['value']))
        except:
            continue
    print('结束更新映射内容到数据库中')
btn = widgets.Button(description='更新映射内容到数据库中')
btn.on_click(_fn)
display(btn)

def _fn(e):
    print('关闭数据库连接')
    cursor.close()
    db.close()
btn = widgets.Button(description='关闭数据库连接')
btn.on_click(_fn)
display(btn)

共有 326 个待映射的名称
[1/326]: 上海体育职业学院
[2/326]: 上海兴韦信息技术职业学院
[3/326]: 上海兴韦信息技术职业学院(原上海托普信息技术职业学院)
[4/326]: 上海医疗器械高等专科学校
[5/326]: 上海对外贸易学院
[6/326]: 上海工商外国语职业学院
[7/326]: 上海工商职业技术学院(原上海新侨职业技术学院)
[8/326]: 上海欧华职业技术学院
[9/326]: 东北石油大学华瑞学院
[10/326]: 东北财经大学津桥商学院
[11/326]: 中原工学院(软件类)
[12/326]: 中国地质大学江城学院
[13/326]: 中国矿业大学
[14/326]: 中国矿业大学(较高收费)
[15/326]: 云南热带作物职业学院
[16/326]: 云南经济管理职业学院
[17/326]: 信阳农业高等专科学校
[18/326]: 信阳农业高等专科学校(中外合作办学)
[19/326]: 信阳农业高等专科学校(对外合作办学)
[20/326]: 信阳师范学院与信阳农业高等专科学校联办
[21/326]: 健雄职业技术学院
[22/326]: 兰州工业高等专科学校
[23/326]: 内蒙古医学院
[24/326]: 内蒙古财经学院
[25/326]: 北京交通大学(威海校区)(较高收费)
[26/326]: 北京化工大学北方学院
[27/326]: 北京吉利大学
[28/326]: 北京建筑工程学院
[29/326]: 北京新圆明职业学院
[30/326]: 北京航空航天大学北海学院
[31/326]: 北海艺术设计职业学院
[32/326]: 十堰职业技术学院
[33/326]: 华中科技大学文华学院
[34/326]: 华北水利水电大学(中外合作办学)各专业入学地点以录取通知书为准
[35/326]: 华北水利水电大学与嵩山少林武术职业学院联办
[36/326]: 华北水利水电大学与嵩山少林武术职业学院联办(前两年就读在嵩山)
[37/326]: 华北水利水电大学与嵩山少林武术职业学院联办(前两年就读嵩山)
[38/326]: 华北水利水电学院
[39/326]: 华北水利水电学院(中外合作办学)
[40/326]: 华北水利水电学院(办学地点在外国语学院)
[41/326]: 华北水利水电学院(办